In [ ]:
import numpy as np
from skimage import io
from skimage.feature import graycomatrix, graycoprops
import math
import pandas as pd
from matplotlib import pyplot as plt

In [ ]:
happy = ['0105', '0107', '0110']
neutral = ['0101', '0106', '0109']
sad = ['0106', '0107', '0110']

selected = {
    'happy': happy,
    'neutral': neutral,
    'sad': sad
}

In [ ]:
def create_pixel_matrix(emotion: str, number_images: list) -> list:
        return io.imread(f'datasets/{emotion}/{emotion}-{number_images}.jpg', as_gray = True)

In [ ]:
def show_histogram(title: str, image_pixel: list):
        plt.subplot(1, 2, 1)
        plt.imshow(image_pixel, cmap = 'gray')
        plt.title(title)

        plt.subplot(1, 2, 2)
        plt.hist(image_pixel.ravel(), 256, [0, 255])
        plt.title(f'{title} histogram')

        plt.show()

In [ ]:
def first_order_statistics(image_pixel: list) -> tuple:
    hist, _ = np.histogram(image_pixel, 256, [0, 255])

    # mean
    # print(np.mean(image_pixel))
    hist_sum = sum(hist)
    hist_per_sum = []
    mean = 0
    for h in hist:
        hist_per_sum.append(h / hist_sum)
    for i in range(256):
        mean += hist_per_sum[i] * i

    # print(mean)

    # variance
    # print(np.var(image_pixel))
    variance = 0
    for i in range(256):
        variance += (i - mean) ** 2 * hist_per_sum[i]
    # print(variance)

    # skewness
    skewness = 0
    for i in range(256):
        skewness += (i - mean) ** 3 * hist_per_sum[i]
    skewness = skewness / (variance * math.sqrt(variance))
    # print(skewness)

    # kurtosis
    kurtosis = 0
    for i in range(256):
        kurtosis += (i - mean) ** 4 * hist_per_sum[i]
    kurtosis = kurtosis / (variance ** 2) - 3
    # print(kurtosis)

    # entropy
    entropy = 0
    for i in range(246):
        entropy += hist_per_sum[i] * math.log2(np.spacing(hist_per_sum[i]))
    entropy = -entropy
    # print(entropy)

    return (mean, variance, skewness, kurtosis, entropy)

In [ ]:
first_statistics = []
image_label = []
pixel_images = {
    'happy': [],
    'neutral': [],
    'sad': []
}

for emotion in ['happy', 'neutral', 'sad']:
    for e in selected[emotion]:
        image_label.append(f'{emotion}-{e}')
        image_pixel = create_pixel_matrix(emotion, e)
        pixel_images[emotion].append(image_pixel)
        show_histogram(f'{emotion}-{e}', image_pixel)
        first_statistics.append(first_order_statistics(image_pixel))

test = pd.DataFrame(first_statistics, columns=['mean', 'variance', 'skewness', 'kurtosis', 'entropy'], index=image_label)
print('First Order Statistics')
test

In [ ]:
def glcm_calculation(pixel_image, dist = [1], angles = [0, np.pi / 4, np.pi / 3, 3 * np.pi / 4], sym = True, norm = True):
    glcm = graycomatrix(pixel_image,
                        distances = dist,
                        angles = angles,
                        symmetric = sym,
                        normed = norm)
    return glcm

In [ ]:
def glcm_features(glcm_matrix: list, props: list) -> list:
    features = []
    # glcm_props = [property for name in props for property in graycoprops(glcm_matrix, name)[0]]
    for name in props:
        for property in graycoprops(glcm_matrix, name):
            features.append(property)

    return np.array(features).ravel()

In [ ]:
def texture_histogram(title: str, glcm_matrix: list):
    for i, angle in enumerate(['0', '45', '90', '135']):
        plt.subplot(1, 4, i + 1)
        plt.hist(glcm_matrix[:,:,0,i])
        plt.title(f'{title}_{angle}', fontsize = 10)
        plt.tight_layout()
    plt.show()

In [ ]:
for emotion in ['happy', 'neutral', 'sad']:
    for i, e in enumerate(selected[emotion]):
        texture_histogram(f'{emotion}-{e}' ,glcm_calculation(pixel_images[emotion][i]))

In [ ]:
properties = ['dissimilarity', 'correlation', 'homogeneity', 'contrast', 'ASM', 'energy']
features_result = []

for emotion in ['happy', 'neutral', 'sad']:
    for pi in pixel_images[emotion]:
        features_result.append(glcm_features(glcm_calculation(pi), properties))

In [ ]:
angles = ['0', '45', '90', '135']
labels = []
index = []
for property in properties:
    for angle in angles:
        labels.append(f'{property}_{angle}')

for emotion in ['happy', 'neutral', 'sad']:
    for e in selected[emotion]:
        index.append(f'{emotion}-{e}')

In [ ]:
second_statistics = pd.DataFrame(features_result, index = index, columns = labels)
second_statistics